In [ ]:
#the basics
import numpy as np
import pandas as pd

#modeling tools
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, confusion_matrix, f1_score

#My modules
import wrangle
import utils

### Acquire Data